In [ ]:
# !pip install --upgrade --quiet langchain langchain-google-genai pandas numpy matplotlib seaborn jupyter openai
# !pip install --upgrade  langchain-openai openai 
# !pip install langchain_community


In [1]:
import csv
import os
import time
import subprocess
import openai
from pathlib import Path
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chat_models import ChatOpenAI


Google

In [ ]:
llm_models = ["gemini-1.0-pro", "gemini-1.5-pro", "gemini-1.5-flash"]
llm_model = llm_models[1]
google_api_key = ""
temperature = 0
input_filename = "../data/leetcode_problems.csv"
output_directory = f"output2/temperature-{temperature}/{llm_model}/"
results_directory = f"results2/temperature-{temperature}/"
time_limit = 60

# Ensure output and results directories exist
Path(output_directory).mkdir(parents=True, exist_ok=True)
Path(results_directory).mkdir(parents=True, exist_ok=True)

# Initialize the chat model
chat = ChatGoogleGenerativeAI(temperature=temperature, google_api_key=google_api_key, model=llm_model)

Openai

In [2]:
# Model settings
llm_models = ["gpt-3.5-turbo", "gpt-4o-mini", "gpt-4o", "gpt-4-turbo"]
llm_model = llm_models[3]
temperature = 1
time_limit = 60

# File and directory paths
input_filename = "../data/leetcode_problems.csv"
output_directory = f"output2/temperature-{temperature}/{llm_model}/"
results_directory = f"results2/temperature-{temperature}/"

# Ensure output and results directories exist
Path(output_directory).mkdir(parents=True, exist_ok=True)
Path(results_directory).mkdir(parents=True, exist_ok=True)

# Set up the OpenAI API key
openai_api_key = ""

# Initialize the chat model with LangChain
chat = ChatOpenAI(api_key=openai_api_key, model_name=llm_model, temperature=temperature)


c:\Users\mkoro\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
response_schemas = [
    ResponseSchema(
        name="problem_solution",
        description="Code a solution in Python for the given problem. "
            "The solution should handle example inputs and verify if each output matches the expected result. "
            "Print for each input the value `True` if the output is correct for that input, otherwise `False`. "
            "Retun the solution as a Python program. "
    ),
    ResponseSchema(
        name="input_1",
        description="Extract the first example input from the problem description. "
                    "Output it formatted as input in a Python program. "
                    "Do not answer if this information is not found."
    ),
    ResponseSchema(
        name="output_1",
        description="Extract the first example output for comparison in a Python program. "
                    "Do not answer if this information is not found."
    ),
    ResponseSchema(
        name="input_2",
        description="Extract the second example input from the problem description. "
                    "Output it formatted as input in a Python program. "
                    "Do not answer if this information is not found."
    ),
    ResponseSchema(
        name="output_2",
        description="Extract the second example output for comparison in a Python program. "
                    "Do not answer if this information is not found."
    ),
    ResponseSchema(
        name="input_3",
        description="Extract the third example input from the problem description. "
                    "Output it formatted as input in a Python program. "
                    "Do not answer if this information is not found."
    ),
    ResponseSchema(
        name="output_3",
        description="Extract the third example output for comparison in a Python program. "
                    "Do not answer if this information is not found."
    ),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# Define the prompt template
solution_template = """
For the following problem description, extract the following information and format it as JSON:

- "Input 1"
- "Output 1"
- "Input 2"
- "Output 2"
- "Input 3"
- "Output 3"
- "Problem Solution"

Text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=solution_template)

Extract the problems

In [4]:
# List to store problems in dictionary format
problems = []

In [5]:
# Read data from the input CSV file
with open(input_filename, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    
    # Skip the header if needed
    next(csv_reader)  # Skip the first line if it's a header
    
    # Process each row in the CSV file
    for fields in csv_reader:
        # Create a dictionary for this problem
        problem = {
            "ID": fields[0],
            "Description": fields[2],
        }
        
        # Add the problem dictionary to the list
        problems.append(problem)


Run to solve the problems

In [6]:
# Initialize an empty list to store data
datos = []

In [7]:
print("Datos: {}".format(datos))

Datos: []


In [8]:
# Define the starting lap and the maximum number of problems to solve
start_lap = 0 # Start from the (startlap + 1) problem
max_problems = 25 # Maximum number of problems to solve

In [9]:

# Limit the number of problems to solve
for lap in range(start_lap, min(len(problems), start_lap + max_problems)):
    try:
        problem_text = problems[lap]["Description"]
            
        # Format messages for chat prompt
        messages = prompt.format_messages(text=problem_text, 
                                          format_instructions=format_instructions)
        
        # Chat with the system and get a response
        response = chat(messages)

        # Take the time before working on the response
        time_start = time.time()

        # Parse the response content using `output_parser`
        output_dict = output_parser.parse(response.content)
        
        
        # Extract the code snippet from the response
        python_code = output_dict.get("problem_solution")
            
        # Process the code to omit the first and last lines
        if "python" in python_code:
            code_lines = python_code.split('\n')[1:-1]
            python_code = '\n'.join(code_lines)
        
        # Save the processed code to a .py file
        output_filename = f"{output_directory}output_{lap + 1}.py"
        with open(output_filename, mode='w', newline='', encoding='utf-8') as pyfile:
            pyfile.write(python_code)
        
        print(f"Python code saved successfully in {output_filename}.")
        
        # Execute the script and capture the output
        script_path = output_filename

        try:
            # Start the subprocess without the timeout
            proceso = subprocess.Popen(["python", script_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            # Use communicate with the timeout
            salida, _ = proceso.communicate(timeout=time_limit)  # Apply the timeout here
            
            resultado = salida.decode().strip()
            
            # Count occurrences of "True" and "False" in the output
            ocurrencias_true = resultado.count("True")
            ocurrencias_false = resultado.count("False")
            
            # Append data to the list
            datos.append({
                "ID": lap + 1,
                "code": python_code,
                "result": resultado,
                "true_count": ocurrencias_true,
                "false_count": ocurrencias_false
            })
        except subprocess.TimeoutExpired:
            proceso.kill()
            datos.append({
                "ID": lap + 1,
                "code": python_code,
                "result": "TIMEOUT",
                "true_count": "TIMEOUT",
                "false_count": "TIMEOUT"
            })

        except (SyntaxError, IndentationError, TypeError, ValueError, ImportError, AttributeError, KeyError, IndexError) as e:
            # Handle specific exceptions separately
            datos.append({
                "ID": lap + 1,
                "code": python_code,  # Include the problematic code
                "result": str(e),
                "true_count": "ERROR",
                "false_count": "ERROR"
            })

        # Take the time after working on the response
        time_end = time.time()
        # Calculate the time elapsed
        time_elapsed = time_end - time_start
    
    except Exception as e:
        datos.append({
            "ID": lap + 1,
            "code": python_code,
            "result": str(e),
            "true_count": "ERROR",
            "false_count": "ERROR"
        })
        
    # Pause execution for 15 seconds before next iteration
    if lap < start_lap + max_problems - 1:
        try:
            if time_elapsed < 3:
                time.sleep(3 - time_elapsed)
        except NameError:
            time.sleep(3)

c:\Users\mkoro\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_1.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_2.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_3.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_4.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_5.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_6.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_7.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_8.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_9.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_10.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_11.py.
Python code saved successfully in output2/temperature-1/gpt-4-turbo/output_12.py.
Python code saved success

In [10]:
print("Response: {}".format(response))

Response: content='```json\n{\n    "problem_solution": "class ListNode:\\n    def __init__(self, val=0, next=None):\\n        self.val = val\\n        self.next = next\\n\\ndef reverseKGroup(head: ListNode, k: int) -> ListNode:\\n    if head is None or k == 1:\\n        return head\\n    dummy = ListNode(0)\\n    dummy.next = head\\n    cur, prev, next = head, dummy, None\\n    count = 0\\n    while cur:\\n        cur = cur.next\\n        count += 1\\n    while count >= k:\\n        cur = prev.next\\n        next = cur.next\\n        for i in range(1, k):\\n            cur.next = next.next\\n            next.next = prev.next\\n            prev.next = next\\n            next = cur.next\\n        prev = cur\\n        count -= k\\n    return dummy.next\\n\\n# testing expected outputs\\nexample_1 = [1, 2, 3, 4, 5]\\nk_1 = 2\\nexpected_1 = [2, 1, 4, 3, 5]\\nexample_2 = [1, 2, 3, 4, 5]\\nk_2 = 3\\nexpected_2 = [3, 2, 1, 4, 5]\\nexample_3 = [1, 2, 3, 4, 5]\\nk_3 = 1\\nexpected_3 = [1, 2, 3, 4

In [11]:
print("Message: {}".format(messages))

Message: [HumanMessage(content='\nFor the following problem description, extract the following information and format it as JSON:\n\n- "Input 1"\n- "Output 1"\n- "Input 2"\n- "Output 2"\n- "Input 3"\n- "Output 3"\n- "Problem Solution"\n\nText: Given a linked list, reverse the nodes of a linked list k at a time and return its modified list.\n\nk is a positive integer and is less than or equal to the length of the linked list. If the number of nodes is not a multiple of k then left-out nodes, in the end, should remain as it is.\n\nFollow up:\nCould you solve the problem in `O(1)` extra memory space?\nYou may not alter the values in the list\'s nodes, only nodes itself may be changed.\n\n\nExample 1:\nInput: head = [1,2,3,4,5], k = 2\nOutput: [2,1,4,3,5]\n\nExample 2:\nInput: head = [1,2,3,4,5], k = 3\nOutput: [3,2,1,4,5]\n\nExample 3:\nInput: head = [1,2,3,4,5], k = 1\nOutput: [1,2,3,4,5]\n\nExample 4:\nInput: head = [1], k = 1\nOutput: [1]\n\nConstraints:\nThe number of nodes in the lis

In [12]:
print("output_dict: {}".format(output_dict))

output_dict: {'problem_solution': 'class ListNode:\n    def __init__(self, val=0, next=None):\n        self.val = val\n        self.next = next\n\ndef reverseKGroup(head: ListNode, k: int) -> ListNode:\n    if head is None or k == 1:\n        return head\n    dummy = ListNode(0)\n    dummy.next = head\n    cur, prev, next = head, dummy, None\n    count = 0\n    while cur:\n        cur = cur.next\n        count += 1\n    while count >= k:\n        cur = prev.next\n        next = cur.next\n        for i in range(1, k):\n            cur.next = next.next\n            next.next = prev.next\n            prev.next = next\n            next = cur.next\n        prev = cur\n        count -= k\n    return dummy.next\n\n# testing expected outputs\nexample_1 = [1, 2, 3, 4, 5]\nk_1 = 2\nexpected_1 = [2, 1, 4, 3, 5]\nexample_2 = [1, 2, 3, 4, 5]\nk_2 = 3\nexpected_2 = [3, 2, 1, 4, 5]\nexample_3 = [1, 2, 3, 4, 5]\nk_3 = 1\nexpected_3 = [1, 2, 3, 4, 5]\n\nprint(reverseKGroup(example_1, k_1) == expected_1

In [13]:
# Write results to a CSV file

nombre_archivo = f"{results_directory}results_{llm_model}.csv"
encabezados = ["ID", "code", "result", "true_count", "false_count"]

with open(nombre_archivo, mode='a', newline='') as archivo_csv:
    escritor_csv = csv.DictWriter(archivo_csv, fieldnames=encabezados)
    if archivo_csv.tell() == 0:
        escritor_csv.writeheader()
    escritor_csv.writerows(datos)

print(f"CSV file '{nombre_archivo}' created successfully.")

CSV file 'results2/temperature-1/results_gpt-4-turbo.csv' created successfully.
